# Attenuated RHESSI example {[Download data files here](https://drive.google.com/file/d/1FMRVh8E0jCh3mj15r_Fy8g_RhbmlGIj-)}
This is looking at the M9 class flare observed by RHESSI from [Knuth+Glesener 2020](https://iopscience.iop.org/article/10.3847/1538-4357/abb779).
We perform spectroscopy on the interval where the thick attenuator is inserted.


## Important note on systematic error
- Systematic error is important to add to RHESSI data so that the minimizer has some wiggle room.

In [ ]:
from sunkit_spex.legacy.fitting_legacy import fitter
from sunkit_spex.extern import rhessi

import astropy.time as atime
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### Notice the warning about attenuator changes!

In [ ]:
spec_fn = 'rhessi-data/rhessi-2011-jul-stixbins-spec.fits'
srm_fn = spec_fn.replace('spec', 'srm')
rl = rhessi.RhessiLoader(spectrum_fn=spec_fn, srm_fn=srm_fn)

### Notice there is no warning when the fit interval doesn't cover an attenuator change!

In [ ]:
rl.update_event_times(atime.Time('2011-07-30T02:08:20'), atime.Time('2011-07-30T02:10:20'))
end_background_time = '2011-07-30T01:56:00'
start_background_time = '2011-07-30T01:54:00'
rl.update_background_times(atime.Time(start_background_time), atime.Time(end_background_time))

In [ ]:
plt.figure()
rl.lightcurve(energy_ranges=[[4, 10], [10, 30], [30, 100]])

### Add systematic error before passing to the fitter object

In [ ]:
# Uniform 10%
rl.systematic_error = 0.1

In [ ]:
ss = fitter.Fitter(rl)
ss.energy_fitting_range = [5, 70]

In [ ]:
plt.figure(layout='constrained')
axs, *_ = ss.plot()
_ = axs[0].set(xscale='log')

### Fit the spectrum with a double thick target electron power law

In [ ]:
def double_thick(electron_flux, low_index, break_energy, up_index, low_cutoff, up_cutoff, energies=None):
    from sunkit_spex.emission import bremsstrahlung_thick_target
    mids = np.mean(energies, axis=1)
    flux = bremsstrahlung_thick_target(
        photon_energies=mids,
        p=low_index,
        eebrk=break_energy,
        q=up_index,
        eelow=low_cutoff,
        eehigh=up_cutoff,
    )
    
    # scale to good units
    return 1e35 * electron_flux * flux

In [ ]:
ss.add_photon_model(double_thick, overwrite=True)

In [ ]:
ss.loglikelihood = 'chi2'
ss.model = 'f_vth + double_thick'

In [ ]:
th_params = [
    'T1_spectrum1',
    'EM1_spectrum1',
]
nth_params = [
    'electron_flux1_spectrum1',
    'low_index1_spectrum1',
    'up_index1_spectrum1',
    'break_energy1_spectrum1',
    'low_cutoff1_spectrum1',
    'up_cutoff1_spectrum1',
]

In [ ]:
ss.params['T1_spectrum1'] = ['free', 20, (5, 100)]
ss.params['EM1_spectrum1'] = ['free', 5000, (500, 100000)]

ss.params['electron_flux1_spectrum1'] = ['free', 10, (1, 50)]
ss.params['low_index1_spectrum1'] = ['free', 5, (1, 20)]
ss.params['up_index1_spectrum1'] = ['free', 5, (1, 20)]

ss.params['break_energy1_spectrum1'] = ['free', 40, (40, 100)]
ss.params['low_cutoff1_spectrum1'] = ['free', 20, (5, 39)]
ss.params['up_cutoff1_spectrum1'] = ['frozen', 500, (5, 1000)]

### Only thermal params vary first

In [ ]:
for p in th_params:
    ss.params[p] = 'free'
for p in nth_params:
    ss.params[p] = 'frozen'

In [ ]:
_ = ss.fit()

### Only nonthermal params vary now

In [ ]:
for p in th_params:
    ss.params[p] = 'frozen'
for p in nth_params:
    ss.params[p] = 'free'

In [ ]:
_ = ss.fit()

### All params are free to vary

In [ ]:
for p in th_params + nth_params:
    ss.params[p] = 'free'

In [ ]:
_ = ss.fit()

In [ ]:
plt.figure(layout='constrained')
ss.plot()
plt.gca().set(xscale='log')
None

### Now you can run MCMC if you want

In [ ]:
# ss.run_mcmc()